<a href="https://colab.research.google.com/github/lsh4205/Discrete_Event_Simulation/blob/main/discrete_event_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Load Data

In [19]:
from google.colab import files
import pandas as pd

##A) Load *trip_stats.csv*

In [5]:
# trip_stats.csv - Use for q_i,j
trip_stats = files.upload()

Saving trip_stats.csv to trip_stats.csv


##B) Load *start_station_probs.csv*

In [6]:
# The start_station_probs.csv file, which you should use
# for the p_i parameter in your simulations.
start_station_probs = files.upload()

Saving start_station_probs.csv to start_station_probs.csv


##C) Read data

In [20]:
df1 = pd.read_csv(list(trip_stats.keys())[0])
df1_csv = pd.read_csv('trip_stats.csv')
df2 = pd.read_csv(list(start_station_probs.keys())[0])
df2_csv = pd.read_csv('start_station_probs.csv')

In [21]:
# trip_stats.csv - Use for q_i,j 
# (probability of station 'j' selected
# as destination from 'i' station)
print(df1)

                      start                                end  count  \
0     11 St & Washington St              11 St & Washington St    142   
1     11 St & Washington St               12 St & Sinatra Dr N     44   
2     11 St & Washington St  14 St Ferry - 14 St & Shipyard Ln     48   
3     11 St & Washington St                    4 St & Grand St     47   
4     11 St & Washington St                    6 St & Grand St     25   
...                     ...                                ...    ...   
5141     Willow Ave & 12 St         Stevens - River Ter & 6 St     23   
5142     Willow Ave & 12 St           Vesey Pl & River Terrace      1   
5143     Willow Ave & 12 St                          Warren St      7   
5144     Willow Ave & 12 St                      Washington St     11   
5145     Willow Ave & 12 St                 Willow Ave & 12 St     50   

           mean         std  
0     25.929108   39.186350  
1     56.655303  149.709313  
2     12.481597   16.335279  
3  

In [22]:
# start_station_porbs.csv - Use for p_i
# (probability of station 'i' selected
# as starting station) 
print(df2_csv)

                              start_station_name  probability
0   South Waterfront Walkway - Sinatra Dr & 1 St     0.044679
1                                  Grove St PATH     0.043504
2       Hoboken Terminal - Hudson St & Hudson Pl     0.033629
3        Hoboken Terminal - River St & Hudson Pl     0.029832
4                                   Newport Pkwy     0.027035
..                                           ...          ...
76                                        Dey St     0.002670
77                                Jackson Square     0.001816
78                       Bergen Ave & Stegman St     0.001457
79                            Grant Ave & MLK Dr     0.000689
80                                    JCBS Depot     0.000010

[81 rows x 2 columns]


# Initial Setup

## A) Set $p_{i}$
Starting-station probabilities
```
df2_csv 
# it will hold the probability of 'start' station.
```

## B) Set $p_{i,j}$
End station probabilities from start station
```
prob_ij[] 
# each row has ['start', 'end', 'probability']
```


- The probability of selecting station $j$, given that the rider is at station $i$, is given by $q_{i,j}$. It is possible that $i = j$, that is, the rider uses the bike but ends up returning it to the same station.

`start_counts ` will hold the total counts of each 'start' station value. 

In [23]:
# Calculate the total counts of each "start" value
start_counts = df1_csv.groupby("start")["count"].sum()
print(start_counts)

start
11 St & Washington St                1817
12 St & Sinatra Dr N                 2515
14 St Ferry - 14 St & Shipyard Ln    2226
4 St & Grand St                      1143
5 Corners Library                     372
                                     ... 
Union St                              281
Van Vorst Park                       1294
Warren St                            1504
Washington St                        1778
Willow Ave & 12 St                   1029
Name: count, Length: 81, dtype: int64




> * First, merge the data from the additional CSV file with the original one. Next, calculate the probability of $p_{start, end}$ by dividing the total counts of the 'start' station by the count of the 'end' station.
* `prob_ij` will hold the `[start, end, probabiltiy]` in each row.



In [24]:
# Merge the total counts with the original CSV file
merged_df = pd.merge(df1_csv, start_counts, on="start", suffixes=("", "_total"))

# Calculate the conditional probabilities of each "end" value given each "start" value
merged_df["probability"] = merged_df["count"] / merged_df["count_total"]

# Select the "start", "end", and "probability" columns
prob_ij = merged_df[["start", "end", "probability"]]

# Print the resulting DataFrame
print(prob_ij)

                      start                                end  probability
0     11 St & Washington St              11 St & Washington St     0.078151
1     11 St & Washington St               12 St & Sinatra Dr N     0.024216
2     11 St & Washington St  14 St Ferry - 14 St & Shipyard Ln     0.026417
3     11 St & Washington St                    4 St & Grand St     0.025867
4     11 St & Washington St                    6 St & Grand St     0.013759
...                     ...                                ...          ...
5141     Willow Ave & 12 St         Stevens - River Ter & 6 St     0.022352
5142     Willow Ave & 12 St           Vesey Pl & River Terrace     0.000972
5143     Willow Ave & 12 St                          Warren St     0.006803
5144     Willow Ave & 12 St                      Washington St     0.010690
5145     Willow Ave & 12 St                 Willow Ave & 12 St     0.048591

[5146 rows x 3 columns]


# Simulation

## Simulation Description

*   Model one logical-day (24 logical-hours) of simulation time.

*   There are some number of stations, m. Each station is modeled, conceptually, as an elementary queue.
*   On a given day, suppose there are n riders in total.
*   The n riders arrive randomly. Their interarrival times constitute an autonomous, stationary, and independent stochastic process, distributed exponentially with mean rate $λ$.
*   When a rider arrives, she selects a bike station randomly. The probability of picking station $i$ is $p_i$.
*   The rider goes to station $i$. If a bike is available, she takes it. Otherwise, she must wait until a bike is available there. Allow for the possibility of an unlimited number of bikes at the station, which will be needed by one of the experiments below.
*   The rider chooses their destination randomly. The probability of selecting station $j$, given that the rider is at station $i$, is given by $q_{i,j}$. It is possible that $i = j$, that is, the rider uses the bike but ends up returning it to the same station.
*   The rider uses the bike for some amount of time before returning it. The amount of time she uses the bike is drawn from a log- normal distribution with mean μ and standard deviation $σ$.
*  After returning her bike, the rider leaves the system.

## A Baseline Experiment
> Suppose the number of riders $n = 3,500$ and through the simulation we will estimate the following:

1. the “probability of a successful rental,” that is, the fraction of riders who are able to get a bike;
2. a rider’s average waiting-time for a bike, considering only riders who successfully got a bike.


Compute a $90%$ confidence interval for your estimates. For the simulation parameters, use the following:

>• For the system-wide interarrival times, which are exponentially distributed, use $λ = 2.38$ riders per minute.

>• For the ride times, which are log-normally distributed, use $μ = 2.78$ and $σ = 0.619$. (This value corresponds with an average ride- time of $e^μ ≈ 16$ minutes.)

>• The $p_i$ values are given in the data file, **start_station_probs.csv**.

>• To derive $q_{i,j}$ values, use the raw data of **trip_stats.csv**.

## Code

In [25]:
import numpy as np
import pandas as pd
from collections import deque

def sim():
    # Rider class
    # start_station: Rider's starting station name
    # end_station: Rider's ending station name
    # arrival_time: Rider's using bike for some amount of time before returning
    class Rider:
        def __init__(self, start_station, end_station, ride_time, arrival_time):
            self.start_station = start_station
            self.end_station = end_station
            self.ride_time = ride_time
            self.arrival_time = arrival_time

    # Parameters
    m = len(df2_csv)
    n = 3500
    lmbda = 2.38
    mu = 2.78
    sigma = 0.619
    bikes_per_station = 10

    # Initialize simulation
    np.random.seed()
    # stations: it will hold the number of bikes in each stations that existed in df2_csv.
    stations = {station: bikes_per_station for station in df2_csv['start_station_name']}
    # waiting_times: it will hold the individual waiting times.
    waiting_times = []
    successful_rentals = 0

    # Simulation loop
    current_time = 0
    riders = []
    # waiting_riders: it will hold the 'start_station' as a key and 'current_time' as a value.
    # then it can access the starting waiting time to calculate waiting time.
    waiting_riders = {station: deque() for station in df2_csv['start_station_name']}
    while current_time < 24 * 60:  # 24 hours in minutes
        # Generate interarrival time and ride time
        interarrival_time = np.random.exponential(1 / lmbda)
        ride_time = np.random.lognormal(mu, sigma)

        current_time += interarrival_time
        if current_time >= 24 * 60:
            break

        # Select a station and destination
        start_station = np.random.choice(df2_csv['start_station_name'], p=df2_csv['probability'])
        end_station = np.random.choice(prob_ij.loc[prob_ij['start'] == start_station]['end'],
                                        p=prob_ij.loc[prob_ij['start'] == start_station]['probability'])

        # Rent a bike
        if stations[start_station] > 0:
            successful_rentals += 1
            stations[start_station] -= 1
            rider = Rider(start_station, end_station, ride_time, current_time)
            riders.append(rider)
        else:
            # If there is no available bike,
            # need to append to waiting_riders deque() with current time 
            # to track a waiting time.
            waiting_riders[start_station].append(current_time)

        # Check for returning bikes
        for r in riders:
            r.ride_time -= interarrival_time
            # If rider is time to return a bike to his/her own end_destination.
            if r.ride_time <= 0:
                # Increment bikes only when the destination is in our system.
                if r.end_station in stations:
                    stations[r.end_station] += 1
                riders.remove(r)

                # Assign a bike to a waiting rider
                if r.end_station in stations and len(waiting_riders[r.end_station]) > 0:
                    # Track a waiting time 
                    waiting_time = current_time - waiting_riders[r.end_station].popleft()
                    waiting_times.append(waiting_time)

    # Calculate the results
    probability_successful_rental = successful_rentals / n
    avg_waiting_time = np.mean(waiting_times)
    return probability_successful_rental, avg_waiting_time

### A) One-time simulation

In [26]:
def one_sim():
    probability_successful_rental, avg_waiting_time = sim()
    print(f"Probability of successful rental: {probability_successful_rental}")
    print(f"Average waiting time for a bike: {avg_waiting_time}")
one_sim()

Probability of successful rental: 0.9285714285714286
Average waiting time for a bike: 77.27988910989167


###B) 10-times simulation

In [29]:
def multi_sim():
    # Run the simulation multiple times
    n_runs = 10
    estimations = [sim() for _ in range(n_runs)]
    prob_successful_rental, avg_waiting_time = zip(*estimations)
    conf_interval_prob = np.percentile(prob_successful_rental, [5, 95])
    conf_interval_waiting_time = np.percentile(avg_waiting_time, [5, 95])
    print(f"90% confidence interval for probability of successful rental: {conf_interval_prob}")
    print(f"90% confidence interval for average waiting time: {conf_interval_waiting_time}")
multi_sim()

90% confidence interval for probability of successful rental: [0.91364286 0.95538571]
90% confidence interval for average waiting time: [44.84401602 78.5001424 ]


## Code Explanation


If there are avaialbe bikes in the `start_station`, rider doesn't need to wait. Also, create `Rider` object that will ride certain amount of time, `ride_time`, from `current_time`.
```python
#Rent a bike
if stations[start_station] > 0:
    successful_rentals += 1
    stations[start_station] -= 1
    rider = Rider(start_station, end_station, ride_time, current_time)
    riders.append(rider)
```
If there is no avaiable bike in the `start_station`. Rider should wait for the other Riders who will return after their `ride_time`.

`waiting_riders` will hold `start_station` as a key and `current_time` as values. Therefore, we can calculate the waiting time by `current_time - waiting_riders[#start_station]`.

```python
else:
    # If there is no available bike,
    # need to append to waiting_riders deque() with current time 
    # to track a waiting time.
    waiting_riders[start_station].append(current_time)
```
If `Rider` is time to return the bike and `end_station` is in our system, the number of bikes will be incremented.

```python
# Check for returning bikes
for r in riders:
    r.ride_time -= interarrival_time
    # If rider is time to return a bike to his/her own end_destination.
    if r.ride_time <= 0:
        # Increment bikes only when the destination is in our system.
        if r.end_station in stations:
            stations[r.end_station] += 1
        riders.remove(r)
```
Then, we calculate the waiting time of `Rider` was in `waiting_riders` queue. 
```python
        # Assign a bike to a waiting rider
        if r.end_station in stations and len(waiting_riders[r.end_station]) > 0:
            # Track a waiting time 
            waiting_time = current_time - waiting_riders[r.end_station].popleft()
            waiting_times.append(waiting_time)
```